# Coursera Data Science Capstone Project
### _Importing libraries for use in the project_

In [8]:
import numpy as np # vector data analysis library
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### Bay Area tech companies and startups list uploaded from Github forked from connor11528

In [40]:
# The code was removed by Watson Studio for sharing.

,Employbl Company ID,Company Name,Website,Address 1,City,State,Zip,Latitude,Longitude,Company Description,Thumbnail URL
1080,1083,Switchboard Software,switchboard-software.com,115 Sansome Street FL 7,San Francisco,CA,94104.0,37.791540,-122.401103,Switchboard enables enterprise business teams ...,NaN
1081,1084,Tata Consultancy Services,tcs.com,5201 Great America Pkwy #522,Santa Clara,CA,95054.0,37.406306,-121.976493,Tata Consultancy Services is a global leader i...,NaN
1082,1085,Cognizant,cognizant.com,500 Frank W Burr Blvd,Teaneck,NJ,7666.0,40.873268,-74.006006,See how Cognizant advances digital growth with...,NaN
1083,1086,Infosys,infosys.com,555 Mission St. Suite 1950,San Francisco,CA,94105.0,37.788690,-122.398695,Infosys is a global leader in next-generation ...,NaN
1084,1087,IBM,ibm.com,555 Bailey Ave,San Jose,CA,95141.0,37.195907,-121.748586,For more than a century IBM has been dedicated...,NaN


### Feature Selection

In [41]:
# Extracting the necessary columns 
df = df[['Company Name','City','Latitude','Longitude']]
df.head()

,Company Name,City,Latitude,Longitude
0,3scale,San Francisco,37.774634,-122.398642
1,8tracks,San Francisco,37.765227,-122.429756
2,10 by 10,San Francisco,37.774929,-122.419415
3,15Five,San Francisco,37.781714,-122.403236
4,21Tech,Oakland,37.803900,-122.270794


In [42]:
# Renaming the columns
df.rename(columns={'Company Name':'Company'}, inplace=True)
df.head()

,Company,City,Latitude,Longitude
0,3scale,San Francisco,37.774634,-122.398642
1,8tracks,San Francisco,37.765227,-122.429756
2,10 by 10,San Francisco,37.774929,-122.419415
3,15Five,San Francisco,37.781714,-122.403236
4,21Tech,Oakland,37.803900,-122.270794


In [43]:
df.shape

(1085, 4)

### Plotting offcies on the map

In [14]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [121]:
# create map of Bay Area using latitude and longitude values
map_sv = folium.Map(location=[37.7749, -122.4194], zoom_start=10)

# add markers to map
for lat, lng, company, city in zip(df['Latitude'], df['Longitude'], df['Company'], df['City']):
    label = '{}, {}'.format(company, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.3,
        parse_html=False).add_to(map_sv)  
    
map_sv

### Using the Foursquare API to explore the regions near the company offices.

In [70]:
CLIENT_ID = 'ZCXRDBKH1GWPDXNAFG2QZAZ2ZXRIG1HQTEAXVOW2T5XCFZHF' # your Foursquare ID
CLIENT_SECRET = 'XKAAED053NP0FRSTM4YDJ0HUV4WDZTFJAGG5DTD0NA1HQHAE' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZCXRDBKH1GWPDXNAFG2QZAZ2ZXRIG1HQTEAXVOW2T5XCFZHF
CLIENT_SECRET:XKAAED053NP0FRSTM4YDJ0HUV4WDZTFJAGG5DTD0NA1HQHAE


In [ ]:
radius = 250
LIMIT = 30

venues = []
for lat, lng, company, city in zip(df['Latitude'], df['Longitude'], df['Company'], df['City']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    
# make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
     # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            lat,
            lng,
            company, 
            city, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [74]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Latitude', 'Longitude', 'Company','City', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(20)

(7271, 8)


,Latitude,Longitude,Company,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,37.774634,-122.398642,3scale,San Francisco,SF Flower Mart,37.775435,-122.400913,Flower Shop
1,37.774634,-122.398642,3scale,San Francisco,Bay Club SF Tennis,37.776571,-122.397948,Tennis Court
2,37.774634,-122.398642,3scale,San Francisco,Local Brewing Co.,37.776550,-122.397171,Brewery
3,37.774634,-122.398642,3scale,San Francisco,Blick Art Materials - Campus Location,37.774133,-122.399310,Arts & Crafts Store
4,37.774634,-122.398642,3scale,San Francisco,Flower Market Cafe,37.775057,-122.401115,Coffee Shop
5,37.774634,-122.398642,3scale,San Francisco,Eviva Mission Bay Fitness Center,37.773056,-122.397292,Gym / Fitness Center
6,37.774634,-122.398642,3scale,San Francisco,Belnacia's Flowers & Cargo,37.775457,-122.401074,Flower Shop
7,37.765227,-122.429756,8tracks,San Francisco,Thorough Bread and Pastry,37.766408,-122.429051,Bakery
8,37.765227,-122.429756,8tracks,San Francisco,Swedish American Hall,37.766756,-122.430563,Concert Hall
9,37.765227,-122.429756,8tracks,San Francisco,The Viking Room,37.766685,-122.430210,New American Restaurant


In [75]:
# Extracting the rows with Venue Category as Gyms
df1 = venues_df.loc[venues_df['VenueCategory'].str.contains('Gym') == True]
df1.head(15)

,Latitude,Longitude,Company,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
5,37.774634,-122.398642,3scale,San Francisco,Eviva Mission Bay Fitness Center,37.773056,-122.397292,Gym / Fitness Center
17,37.765227,-122.429756,8tracks,San Francisco,Alex Fitness,37.764689,-122.432183,Gym
45,37.774929,-122.419415,10 by 10,San Francisco,NEMA Fitness Center,37.775700,-122.416942,Gym
55,37.774929,-122.419415,10 by 10,San Francisco,Orangetheory Fitness,37.775826,-122.417252,Gym / Fitness Center
61,37.781714,-122.403236,15Five,San Francisco,Core40,37.780562,-122.403194,Gym / Fitness Center
70,37.781714,-122.403236,15Five,San Francisco,SKYEFiT,37.780973,-122.402667,Gym
130,37.791722,-122.402020,80Twenty,San Francisco,Equinox Pine Street,37.792036,-122.401289,Gym
135,37.791722,-122.402020,80Twenty,San Francisco,Barry's Bootcamp,37.790608,-122.402807,Gym / Fitness Center
137,37.791722,-122.402020,80Twenty,San Francisco,Bay Club - Financial District,37.792443,-122.403999,Gym
142,37.791722,-122.402020,80Twenty,San Francisco,Flagship CrossFit,37.791747,-122.402559,Gym


In [76]:
# Sorting the rows by Venue Category
df1.sort_values(by=['VenueCategory'],inplace=True)
df1.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Latitude,Longitude,Company,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
3768,37.790029,-122.400840,Coinbase,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym
7073,37.758335,-122.412322,Hoodline,San Francisco,Hit Fit SF,37.759795,-122.412651,Boxing Gym
4793,37.791480,-122.400593,Duncan Channon,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym
7029,37.792085,-122.399368,Holberton School,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym
613,37.792085,-122.399368,After College,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym


In [77]:
# Creating Dummy Variables for Venue Category
onehot = pd.get_dummies(df1['VenueCategory'],prefix='',prefix_sep='')

In [78]:
# Creating a copy of data frame
df3 = df1

In [79]:
onehot.head()

,Boxing Gym,Climbing Gym,Gym,Gym / Fitness Center,Gymnastics Gym
3768,1,0,0,0,0
7073,1,0,0,0,0
4793,1,0,0,0,0
7029,1,0,0,0,0
613,1,0,0,0,0


In [87]:
df3.head()

,Latitude,Longitude,Company,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory
3768,37.790029,-122.400840,Coinbase,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym
7073,37.758335,-122.412322,Hoodline,San Francisco,Hit Fit SF,37.759795,-122.412651,Boxing Gym
4793,37.791480,-122.400593,Duncan Channon,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym
7029,37.792085,-122.399368,Holberton School,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym
613,37.792085,-122.399368,After College,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym


In [81]:
# Merging the dummy variables with the original dataframe
df4 = pd.merge(df3,onehot,on=df3.index)

In [82]:
df4.head(30)

,key_0,Latitude,Longitude,Company,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory,Boxing Gym,Climbing Gym,Gym,Gym / Fitness Center,Gymnastics Gym
0,3768,37.790029,-122.400840,Coinbase,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
1,7073,37.758335,-122.412322,Hoodline,San Francisco,Hit Fit SF,37.759795,-122.412651,Boxing Gym,1,0,0,0,0
2,4793,37.791480,-122.400593,Duncan Channon,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
3,7029,37.792085,-122.399368,Holberton School,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
4,613,37.792085,-122.399368,After College,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
5,6406,37.790047,-122.400874,Grammarly,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
6,6126,37.791192,-122.401818,GitLab,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
7,4490,37.760280,-122.411927,DesignMap,San Francisco,Hit Fit SF,37.759795,-122.412651,Boxing Gym,1,0,0,0,0
8,6474,37.790038,-122.419915,Grove Collaborative,San Francisco,Polk Street Boxing Gym,37.790252,-122.420380,Boxing Gym,1,0,0,0,0
9,6897,37.791479,-122.399947,Helpshift,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0


In [83]:
df4.drop(['key_0'],axis=1,inplace=True)

In [86]:
df4.head(100)

,Latitude,Longitude,Company,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory,Boxing Gym,Climbing Gym,Gym,Gym / Fitness Center,Gymnastics Gym
0,37.790029,-122.400840,Coinbase,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
1,37.758335,-122.412322,Hoodline,San Francisco,Hit Fit SF,37.759795,-122.412651,Boxing Gym,1,0,0,0,0
2,37.791480,-122.400593,Duncan Channon,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
3,37.792085,-122.399368,Holberton School,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
4,37.792085,-122.399368,After College,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
5,37.790047,-122.400874,Grammarly,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
6,37.791192,-122.401818,GitLab,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
7,37.760280,-122.411927,DesignMap,San Francisco,Hit Fit SF,37.759795,-122.412651,Boxing Gym,1,0,0,0,0
8,37.790038,-122.419915,Grove Collaborative,San Francisco,Polk Street Boxing Gym,37.790252,-122.420380,Boxing Gym,1,0,0,0,0
9,37.791479,-122.399947,Helpshift,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0


In [89]:
group_clustre = df4.drop(['City','Company','Latitude','Longitude','VenueName','VenueLatitude','VenueLongitude','VenueCategory'],axis=1)

In [90]:
group_clustre.head()

,Boxing Gym,Climbing Gym,Gym,Gym / Fitness Center,Gymnastics Gym
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


### Using KMeans algorithm for the clustering of the gyms near companies

In [91]:
k=5
kmeans = KMeans(n_clusters=k,random_state=0).fit(group_clustre)
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [92]:
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [93]:
df4.head()

,Latitude,Longitude,Company,City,VenueName,VenueLatitude,VenueLongitude,VenueCategory,Boxing Gym,Climbing Gym,Gym,Gym / Fitness Center,Gymnastics Gym
0,37.790029,-122.400840,Coinbase,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
1,37.758335,-122.412322,Hoodline,San Francisco,Hit Fit SF,37.759795,-122.412651,Boxing Gym,1,0,0,0,0
2,37.791480,-122.400593,Duncan Channon,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
3,37.792085,-122.399368,Holberton School,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0
4,37.792085,-122.399368,After College,San Francisco,Rumble Boxing - FiDi,37.791941,-122.400706,Boxing Gym,1,0,0,0,0


In [94]:
df4.insert(0,'ClusterLabels',kmeans.labels_)

### Creating map of all gyms along with specialization along with all companies.

In [118]:
# create map

map_clusters = folium.Map(location=[37.7749, -122.4194], zoom_start=14)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, name, cat, cluster in zip(df4['VenueLatitude'], df4['VenueLongitude'], df4['VenueName'],df4['VenueCategory'], df4['ClusterLabels']):
    label = folium.Popup(name +','+ cat + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
    
for lat, lng, company, city in zip(df4['Latitude'], df4['Longitude'], df4['Company'], df4['City']):
    label = '{}, {}'.format(company, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='black',
        fill=True,
        fill_color='black',
        fill_opacity=0.3,
        parse_html=False).add_to(map_clusters)    
map_clusters

### Cluster Analysis

In [104]:
cluster0 = df4.loc[df4['ClusterLabels'] == 0, df4.columns[[1] + list(range(6, df4.shape[1]))]]

### Cluster 0 is shown in Red Colour and represents Traditional Gyms

In [105]:
cluster0['VenueCategory'].value_counts()

Gym    214
Name: VenueCategory, dtype: int64

In [106]:
cluster1 = df4.loc[df4['ClusterLabels'] == 1, df4.columns[[1] + list(range(6, df4.shape[1]))]]

### Cluster 1 is shown in Blue Colour and represents Gyms / Fitness Centers

In [107]:
cluster1['VenueCategory'].value_counts()

Gym / Fitness Center    173
Name: VenueCategory, dtype: int64

In [108]:
cluster3 = df4.loc[df4['ClusterLabels'] == 2, df4.columns[[1] + list(range(6, df4.shape[1]))]]
cluster4 = df4.loc[df4['ClusterLabels'] == 3, df4.columns[[1] + list(range(6, df4.shape[1]))]]
cluster5 = df4.loc[df4['ClusterLabels'] == 4, df4.columns[[1] + list(range(6, df4.shape[1]))]]

### Cluster 3 is shown in Purple color and represents Boxing Gyms

In [109]:
cluster3['VenueCategory'].value_counts()

Boxing Gym    22
Name: VenueCategory, dtype: int64

### Cluster 4 is shown in Orange color and represents Climbing Gyms

In [110]:
cluster4['VenueCategory'].value_counts()

Climbing Gym    2
Name: VenueCategory, dtype: int64

### Cluster 5 is shown in Lime color and represents Gymnastic Gyms

In [111]:
cluster5['VenueCategory'].value_counts()

Gymnastics Gym    3
Name: VenueCategory, dtype: int64